In [143]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import yfinance as yf
yf.pdr_override()

In [3]:
import pandas as pd

pd.set_option('display.max_rows', 10)

df = pd.read_csv('TICKERS.csv')

df.columns = ['empresa','ticker1','ticker2','ticker3']

df = df.melt(id_vars=['empresa'])

df.drop(columns=['variable'],inplace=True)

df.dropna(inplace=True)

df.set_index('empresa',inplace=True)

df.sort_index(key=lambda col: col.str.lower(),inplace=True)



,value
empresa,
3M,MMMC34
3R Petroleum,RRRP3
Abbott Laboratories,ABTT34
Aeris,AERI3
AES Brasil,AESB3
...,...
Whirlpool,WHRL4
Whirpool,WHRL3
Xerox,XRXB34


In [4]:
aux = []

#Carregar os dados

for i in range(df.shape[0]):
    aux.append(web.get_data_yahoo(df.iloc[i,0]+'.sa',start='2021-01-01',end = '2021-04-30'));
    
df.insert(1, 'dados', aux, allow_duplicates=True);

print('======= ACABOU ===========')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [10]:
# salvar data frame

df.to_hdf('b3test.h5', 'obj1');

In [ ]:
# carregar

df = pd.read_hdf('b3test.h5', 'obj1');

In [17]:
# atualização

from datetime import datetime

data = datetime.fromisoformat('1970-01-01')

for i in range(df.shape[0]):
    if (df.iloc[i,1].shape[0]>0):
        if (max(df.iloc[i,1].index) > data):
            data = max(df.iloc[i,1].index)

for i in range(df.shape[0]):
    df2 = df.iloc[i,1].append(web.get_data_yahoo(df.iloc[i,0]+'.sa',start=data))
    df2 = df2[~df2.index.duplicated(keep='first')]
    df.iat[i,1] = df2

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [195]:
#setup 9.1
#calculo da MME9

for i in range(df.shape[0]):
    
    aux[i]['MME9'] = aux[i].Close.ewm(span=9).mean().dropna()
    for k in range (aux[i].shape[0]):
        aux[i]['SLOPEMME9'][k] = aux[i].MME9[k] - aux[i].MME9[k-1]
                

for i in range(df.shape[0]):
    aux[i].SLOPEMME9[0]=0
print('don')                     
            

<ipython-input-195-4e1d02558604>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aux[i]['SLOPEMME9'][k] = aux[i].MME9[k] - aux[i].MME9[k-1]


don


<ipython-input-195-4e1d02558604>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aux[i].SLOPEMME9[0]=0


In [246]:
#perca de minima
for i in range(df.shape[0]):
   aux[i]['mark_min'] = np.where((aux[i].Close < aux[i].shift(1).Low) & (aux[i].SLOPEMME9 > 0) & (aux[i].index > '2021-04-26'),1,0)

In [247]:
#saber quantos mark_min
contador=0

for i in range(df.shape[0]):
    for j in range(aux[i].shape[0]):
        if(aux[i].mark_min[j] !=0):
            contador = contador + 1;
contador

# Achar os mark_min
# aux[0].loc[aux[0]['mark_min'] == 1]

32

In [248]:
#superação de maxima
for i in range(df.shape[0]):
   aux[i]['mark_max'] = np.where(((aux[i].Close > aux[i].shift(1).High) | (aux[i].Open > aux[i].shift(1).High)) & (aux[i].SLOPEMME9 > 0) & ((aux[i].shift(1).mark_min == 1) | (aux[i].shift(2).mark_min == 1) | (aux[i].shift(3).mark_min == 1)) ,1,0)


In [249]:
#saber quantos mark_max
contador=0

for i in range(df.shape[0]):
    for j in range(aux[i].shape[0]):
        if(aux[i].mark_max[j] !=0):
            contador = contador + 1;
contador

7

In [250]:
#pegar todos os tickers com mark_max
tickers = [];
k=0;
for i in range(df.shape[0]):
    for j in range(aux[i].shape[0]):
        if (aux[i]['mark_max'][j] == 1):
            tickers.append(df.iloc[i,0]);
        
            
        

In [251]:
tickers

['CMIG4', 'HGTX3', 'FESA3', 'FESA4', 'K2CG34', 'PCAR3', 'WFCO34']

In [262]:
for i in range(df.shape[0]):
    aux[i].loc[aux[i]['mark_max'] == 1]

In [260]:
aux[0].loc[aux[0]['mark_max'] == 1]

,Open,High,Low,Close,Adj Close,Volume,MME9,SLOPEMME9,mark_min,mark_max
Date,,,,,,,,,,


In [263]:
df.shape[0]

336

In [5]:
import pandas as pd

a = [1,2,3,4]
b = [7,8,9,0]

dd = pd.DataFrame({'a':a, 'b':b})

dd.set_index('a',inplace=True)

dd

,b
a,
1,7
2,8
3,9
4,0
